In [2]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import astropy.coordinates as coord
# import dask.dataframe as dd


In [3]:
with h5py.File('/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced_feh.hdf5', 'r') as f:
    ra = f['ra'][:]
    dec = f['dec'][:]
    parallax = f['parallax'][:]
    pmra = f['pmra'][:]
    pmdec = f['pmdec'][:]
    rv = f['radial_velocity'][:]
    source_id = f['source_id'][:]
    poe = f['parallax_over_error'][:]
    feh = f['feh'][:]
    z = f['z_gal'][:]

df = pd.DataFrame(
{
    #sort based on id
    'source_id': source_id,
    'parallax': parallax,
    'ra': ra,
    'dec': dec,
    'pmra': pmra,
    'pmdec': pmdec,
    'rv': rv,
    'poe': poe,
    'feh': feh,
    'z': z
})

new_df = df[(np.absolute(df['z']) > 1.5) & (df['feh'] < -1.5)]

In [8]:
new_df.source_id.index

Index([    2355,     4409,     4410,     7539,     7540,     8274,     8276,
           8691,     9493,     9494,
       ...
       90127937, 90130757, 90139207, 90139770, 90143463, 90145188, 90152320,
       90153110, 90153770, 90153907],
      dtype='int64', length=33306)

In [5]:
np.sum(~(np.absolute(new_df.z)>1.5)), np.sum(~(new_df.feh<-1.5))

(0, 0)

In [9]:
new_is_accreted = np.empty(len(df.source_id), dtype=bool)
new_is_accreted.fill(False)
new_is_accreted[new_df.source_id.index] = True
# np.isin(df.source_id, new_df.source_id)
new_is_accreted.shape, df.source_id.shape

((90172126,), (90172126,))

In [16]:
new_is_accreted = 

: 

: 

39922

In [10]:
with h5py.File('/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced_feh.hdf5', 'a') as f:
    del f['is_accreted']
    f.create_dataset('is_accreted', data=new_is_accreted)

: 

In [3]:
# path0 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_00.h5'
# path1 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_01.h5'
# path2 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_02.h5'
# path3 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_03.h5'
# path4 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_04.h5'
# path5 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_05.h5'
# path6 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_06.h5'
# path7 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_07.h5'
# path8 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_08.h5'
# path9 = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/stellar_params_catalog_09.h5'

In [4]:
# files = [path0, path1, path2, path3, path4, path5, path6, path7, path8, path9]
# feh = []
# BPRP_id = []
# for i in range(len(files)):
#     with h5py.File(files[i], 'r') as f:
#         feh = np.append(feh, f['stellar_params_est'][:,1])
#         b_id = np.array(f['gdr3_source_id'][:], dtype=np.int64)
#         BPRP_id = np.append(BPRP_id, b_id)
    
# with h5py.File('/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/BPRP_feh.h5', 'w') as f:
#     f.create_dataset('feh', data=feh)
#     f.create_dataset('BPRP_id', data=BPRP_id, dtype='int64')

In [3]:
with h5py.File('/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/BPRP_spec/BPRP_feh.h5', 'r') as f:
    feh = f['feh'][:]
    BPRP_id = f['BPRP_id'][:]

with h5py.File('/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced.hdf5', 'r') as f:
    ra = f['ra'][:]
    dec = f['dec'][:]
    parallax = f['parallax'][:]
    pmra = f['pmra'][:]
    pmdec = f['pmdec'][:]
    rv = f['radial_velocity'][:]
    source_id = f['source_id'][:]
    poe = f['parallax_over_error'][:]

In [5]:
path_feh = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced_feh.hdf5'
with h5py.File(path_feh, 'r') as f:
    print(list(f.keys()))
    ra = f['ra'][:]
    dec = f['dec'][:]
    parallax = f['parallax'][:]
    pmra = f['pmra'][:]
    pmdec = f['pmdec'][:]
    rv = f['radial_velocity'][:]
    source_id = f['source_id'][:]
    feh = f['feh'][:]


['dec', 'feh', 'parallax', 'parallax_over_error', 'pmdec', 'pmra', 'ra', 'radial_velocity', 'source_id']


In [6]:
ra.shape, dec.shape, parallax.shape, pmra.shape, pmdec.shape, rv.shape, source_id.shape, feh.shape, feh

((90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 array([-0.16963588,  0.08896311, -1.05859041, ..., -0.52068067,
        -0.26526949,  0.36386272]))

In [4]:
ra.shape, dec.shape, parallax.shape, pmra.shape, pmdec.shape, rv.shape, source_id.shape, feh.shape, feh

((90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 (90172126,),
 array([-0.16963588,  0.08896311, -1.05859041, ..., -0.52068067,
        -0.26526949,  0.36386272]))

In [4]:
match_idx = np.isin(source_id, BPRP_id, assume_unique=True)

In [1]:
new_source_id = source_id[match_idx]
new_ra = ra[match_idx]
new_dec = dec[match_idx]
new_parallax = parallax[match_idx]
new_pmra = pmra[match_idx]
new_pmdec = pmdec[match_idx]
new_rv = rv[match_idx]
new_poe = poe[match_idx]

NameError: name 'source_id' is not defined

In [11]:
match_idx, np.sum(np.isin(source_id, source_id[match_idx]))

(array([ True,  True,  True, ...,  True,  True,  True]), 88289288)

In [12]:
np.sum(match_idx), len(match_idx)

(84939935, 98346986)

In [4]:
df_bprp = pd.DataFrame(
{
    'feh': feh,
    'BPRP_id': BPRP_id
},
#     index=BPRP_id,
)

n_begin = 0
n_end = 1000

df_gaia = pd.DataFrame(
{
    #sort based on id
    'source_id': source_id[n_begin:n_end],
    'parallax': parallax[n_begin:n_end],
    'ra': ra[n_begin:n_end],
    'dec': dec[n_begin:n_end],
    'pmra': pmra[n_begin:n_end],
    'pmdec': pmdec[n_begin:n_end],
    'rv': rv[n_begin:n_end],
    'poe': poe[n_begin:n_end],
})

# BPRP_ddf = dd.from_pandas(BPRP_df, npartitions=10000000)
# gaia_ddf = dd.from_pandas(gaia_df, npartitions=10000000)

In [1]:
df_gaia

NameError: name 'df_gaia' is not defined

In [6]:
merged_df = df_gaia.merge(df_bprp, left_on='source_id', right_on='BPRP_id')

: 

: 

In [6]:
# Assuming df_gaia and df_bprp are your already loaded DataFrames
chunksize_gaia = 5000  # Define chunk size for df_gaia
chunksize_bprp = 5000  # Define chunk size for df_bprp

def merge_chunks(chunk_gaia, chunk_bprp):
    return pd.merge(chunk_gaia, chunk_bprp, left_on='source_id', right_on='BPRP_id', how='left')

final_result = pd.DataFrame()

# Iterate over chunks of df_gaia
for start_gaia in range(0, df_gaia.shape[0], chunksize_gaia):
    end_gaia = min(start_gaia + chunksize_gaia, df_gaia.shape[0])
    chunk_gaia = df_gaia.iloc[start_gaia:end_gaia]

    # Iterate over chunks of df_bprp
    for start_bprp in range(0, df_bprp.shape[0], chunksize_bprp):
        end_bprp = min(start_bprp + chunksize_bprp, df_bprp.shape[0])
        chunk_bprp = df_bprp.iloc[start_bprp:end_bprp]

        # Merge and accumulate the results
        merged_chunk = merge_chunks(chunk_gaia, chunk_bprp)
        final_result = pd.concat([final_result, merged_chunk], ignore_index=True)

# 'final_result' is your merged DataFrame


: 

: 

In [ ]:

# Assuming each HDF5 file contains a complete DataFrame under a single key
gaia_dataset_key = ['source_id', 'ra', 'dec', 'parallax', 'parallax_over_error', 'pmra', 'pmdec', 'radial_velocity']  # Replace with the actual key for the Gaia dataset
bprp_dataset_key = ['feh', 'BPRP_id']  # Replace with the actual key for the BPRP dataset

# Read the complete DataFrames
df_gaia_full = dd.read_hdf('/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced.hdf5', key=gaia_dataset_key)
df_bprp_full = dd.read_hdf('/ocean/projects/phy210068p/hsu1/BPRP_spec/BPRP_feh.h5', key=bprp_dataset_key)

# Select the required columns from each DataFrame
df_gaia = df_gaia_full[['source_id', 'ra', 'dec', 'parallax', 'parallax_over_error', 'pmra', 'pmdec', 'radial_velocity']]
df_bprp = df_bprp_full[['feh', 'BPRP_id']]

# Perform the merge
merged_df = dd.merge(df_gaia, df_bprp, left_on='source_id', right_on='BPRP_id', how='left')

# Compute the result
result = merged_df.compute()

In [ ]:
path = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced.hdf5'

with h5py.File(path, 'r') as f:
    main_source_id = np.array(f['source_id'][:])
    parallax = np.array(f['parallax'][:])
    ra = np.array(f['ra'][:])
    dec = np.array(f['dec'][:])
    pmra = np.array(f['pmra'][:])
    pmdec = np.array(f['pmdec'][:])
    rv = np.array(f['radial_velocity'][:])
    poe = np.array(f['parallax_over_error'][:])

In [ ]:
gaia_df = pd.DataFrame(
{
    #sort based on id
    'main_source_id': main_source_id[begin:end],
    'parallax': parallax[begin:end],
    'ra': ra[begin:end],
    'dec': dec[begin:end],
    'pmra': pmra[begin:end],
    'pmdec': pmdec[begin:end],
    'rv': rv[begin:end],
    'poe': poe[begin:end],
})

gaia_df = pd.DataFrame(
{
    #sort based on id
    'main_source_id': main_source_id[begin:end],
    'parallax': parallax[begin:end],
    'ra': ra[begin:end],
    'dec': dec[begin:end],
    'pmra': pmra[begin:end],
    'pmdec': pmdec[begin:end],
    'rv': rv[begin:end],
    'poe': poe[begin:end],
})

# merged_df = gaia_df.merge(BPRP_df, left_on='main_source_id', right_on='BPRP_id')
merged_df = df1.join(df2, on='source_id', how='inner')

In [ ]:
begin = 0
end = 100

gaia_df = pd.DataFrame(
{
#sort based on id
'main_source_id': main_source_id,
'parallax': parallax,
'ra': ra,
'dec': dec,
'pmra': pmra,
'pmdec': pmdec,
'rv': rv,
'poe': poe,
})

merged_df = gaia_df.merge(BPRP_df, left_on='main_source_id', right_on='BPRP_id')
total_df = pd.concat([total_df, merged_df], ignore_index=True)

In [ ]:

total_df = pd.DataFrame()
chunksize = 1000000

for chunk in range(chunksize):
    begin = int(chunk * gaia_len / chunksize)
    end = int((chunk + 1) * gaia_len / chunksize)

    gaia_df = pd.DataFrame(
    {
        #sort based on id
        'main_source_id': main_source_id[begin:end],
        'parallax': parallax[begin:end],
        'ra': ra[begin:end],
        'dec': dec[begin:end],
        'pmra': pmra[begin:end],
        'pmdec': pmdec[begin:end],
        'rv': rv[begin:end],
        'poe': poe[begin:end],
    })

    # merged_df = gaia_df.merge(BPRP_df, left_on='main_source_id', right_on='BPRP_id')

    total_df = pd.concat([total_df, merged_df], ignore_index=True)

In [ ]:
ra.shape

In [4]:
gaia_df = pd.DataFrame(
    {
        #sort based on id
        'source_id': source_id,
        'parallax': parallax,
        'ra': ra,
        'dec': dec,
        'pmra': pmra,
        'pmdec': pmdec,
        'rv': rv,
        'parallax_over_error': poe,
    },
#     index= main_source_id,
)

In [5]:
BPRP_df = pd.DataFrame(
    {
        'feh': feh,
        'BPRP_id': BPRP_id
    },
#     index=BPRP_id,
)

In [1]:
BPRP_df, gaia_df

NameError: name 'BPRP_df' is not defined

In [10]:
merged_df = gaia_df.merge(BPRP_df, left_on='source_id', right_on='BPRP_id')

: 

: 

In [9]:
BPRP_id, source_id

(array([         4295806720,         38655544960,       1275606125952, ...,
        6917528649684202496, 6917528890203987968, 6917528993281571840]),
 array([5978968864037824512, 5122066607233746944, 4187730797380655616, ...,
        5961421792352868352, 5961421826761172992, 5961421861120892928]))

In [ ]:
merged_df, merged_df.feh.shape, ra.shape, merged_df.ra.shape

In [ ]:
ra = merged_df.ra
dec = merged_df.dec
parallax = merged_df.parallax
pmra = merged_df.pmra
pmdec = merged_df.pmdec
rv = merged_df.rv
feh = merged_df.feh
source_id = merged_df.main_source_id

with h5py.File('/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced_feh.hdf5', 'a') as f:
    f.create_dataset('ra', data=ra)
    f.create_dataset('dec', data=dec)
    f.create_dataset('parallax', data=parallax)
    f.create_dataset('pmra', data=pmra)
    f.create_dataset('pmdec', data=pmdec)
    f.create_dataset('radial_velocity', data=rv)
    f.create_dataset('feh', data=feh)
    f.create_dataset('source_id', data=source_id)


In [ ]:
with h5py.File('/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced.hdf5', 'r') as f:
    ra = f['ra'][:]

ra.shape

In [ ]:
ra

In [ ]:
merged_df = merged_df.dropna()

In [ ]:
merged_df

In [ ]:
source_id = merged_df.main_source_id
parallax = merged_df.parallax
poe = merged_df.poe
ra = merged_df.ra
dec = merged_df.dec
pmra = merged_df.pmra
pmdec = merged_df.pmdec
rv = merged_df.rv
feh = merged_df.feh


In [ ]:
poe = np.array(poe)
ra = np.array(ra)
dec = np.array(dec)
pmra = np.array(pmra)
pmdec = np.array(pmdec)
rv = np.array(rv)
parallax = np.array(parallax)
feh = np.array(feh)

In [ ]:

ra = ra * u.deg
dec = dec * u.deg
pmra = pmra * u.mas / u.yr
pmdec = pmdec * u.mas / u.yr
parallax = parallax * u.mas
rv = rv * u.km / u.s

dist = coord.Distance(parallax=parallax, allow_negative=True)

# Coord transformation
icrs = coord.ICRS(
    ra=ra, dec=dec, distance=dist, pm_ra_cosdec=pmra, pm_dec=pmdec, radial_velocity=rv)
gal = icrs.transform_to(coord.Galactocentric())
x_gal = gal.x.to_value(u.kpc)
y_gal = gal.y.to_value(u.kpc)
z_gal = gal.z.to_value(u.kpc)
vx_gal = gal.v_x.to_value(u.km/u.s)
vy_gal = gal.v_y.to_value(u.km/u.s)
vz_gal = gal.v_z.to_value(u.km/u.s)  

In [ ]:
non_null_mask = (poe>10)

In [ ]:
rv = rv[non_null_mask]
ra = ra[non_null_mask]
dec = dec[non_null_mask]
parallax = parallax[non_null_mask]
pmra = pmra[non_null_mask]
pmdec = pmdec[non_null_mask]
feh = feh[non_null_mask]
poe = poe[non_null_mask]
source_id = source_id[non_null_mask]

In [ ]:
# path_gaia = '/scratch/08949/tg882489/data/Ananke/GaiaDR3_data_reduced.hdf5'
# with h5py.File(path_gaia, 'w') as f:
#     f.create_dataset('radial_velocity', data=rv)
#     f.create_dataset('ra', data=ra)
#     f.create_dataset('dec', data=dec)
#     f.create_dataset('pmra', data=pmra)
#     f.create_dataset('pmdec', data=pmdec)
#     f.create_dataset('parallax', data=parallax)
#     f.create_dataset('feh', data=feh)
#     f.create_dataset('poe', data=poe)
#     f.create_dataset('source_id', data=source_id)

In [ ]:
path_gaia = '/scratch/08949/tg882489/data/Ananke/GaiaDR3_data_reduced.hdf5'
with h5py.File(path_gaia, 'r') as f:
    rv = f['radial_velocity'][:]
    ra = f['ra'][:]
    dec = f['dec'][:]
    pmra = f['pmra'][:]
    pmdec = f['pmdec'][:]
    parallax = f['parallax'][:]
    feh = f['feh'][:]
    source_id = f['source_id'][:]

In [ ]:
keep_source_id = source_id
empt = np.zeros(len(keep_source_id))

In [ ]:

ra = ra * u.deg
dec = dec * u.deg
pmra = pmra * u.mas / u.yr
pmdec = pmdec * u.mas / u.yr
parallax = parallax * u.mas
rv = rv * u.km / u.s

dist = coord.Distance(parallax=parallax, allow_negative=True)

# Coord transformation
icrs = coord.ICRS(
    ra=ra, dec=dec, distance=dist, pm_ra_cosdec=pmra, pm_dec=pmdec, radial_velocity=rv)
gal = icrs.transform_to(coord.Galactocentric())
x_gal = gal.x.to_value(u.kpc)
y_gal = gal.y.to_value(u.kpc)
z_gal = gal.z.to_value(u.kpc)
vx_gal = gal.v_x.to_value(u.km/u.s)
vy_gal = gal.v_y.to_value(u.km/u.s)
vz_gal = gal.v_z.to_value(u.km/u.s)  

In [ ]:
z_mask = (np.absolute(z_gal) > 1.5)
source_id_z = source_id[z_mask]
vx = vx_gal[z_mask]
vy = vy_gal[z_mask]
vz = vz_gal[z_mask]
feh = feh[z_mask]


In [ ]:
non_null_mask = (feh < -1.5)
source_id = source_id[non_null_mask]
vx = vx[non_null_mask]
vy = vy[non_null_mask]
vz = vz[non_null_mask]
feh = feh_z[non_null_mask]

In [ ]:
len(source_id), len(vx)

In [ ]:
vr = np.sqrt(vx**2+vz**2)

In [ ]:
len(z_gal), len(feh), len(vx), np.sum(np.isnan(source_id))

In [ ]:
len(feh)

In [ ]:
# source_id = str(source_id)
# keep_source_id = str(keep_source_id)

In [ ]:
np.sum(accretion), len(keep_source_id)

In [ ]:
accretion_id = np.isin(keep_source_id, source_id)
accretion = np.int32(accretion_id)
is_accreted = accretion

In [ ]:
with h5py.File(path_gaia, 'a') as f:
    del f['is_accreted']
    f.create_dataset('is_accreted', data=accretion)

In [ ]:
fig, ax = plt.subplots(1, figsize=(13, 10))

h = ax.hist2d(vy, vr, bins=100, weights=np.repeat(1/len(vx), len(vx)), norm=mpl.colors.LogNorm())
ax.set_xlabel(r'$V_y [km s^{-1}]$', fontsize=35)
ax.set_ylabel(r'$\sqrt{V_x^2+V_z^2} [km s^{-1}]$', fontsize=35)
ax.set_title('Toomre Diagram of Accreted Stars', fontsize=40)
cb = fig.colorbar(h[3], ax=ax, label=r'Normalized Counts (km/s)$^{-2}$') 
cb.ax.tick_params(labelsize=35)
ax.tick_params(axis='both', labelsize=35)

In [ ]:
len(vy)